## Lab 2: Tools Deployment, AgentCore Identity, and AgentCore Gateway Integration

### Overview
Lab 1 introduced Strands Agents, tools, and browser/code-interpreter integrations. It demonstrated how to create and test tools locally. In this lab we take the same local tools but deploy it into remote runtimes so agents can call it in production-like settings. We will cover building images (or packaging code), configuring authentication (OAuth2 or IAM), and registering tool endpoints so agents can call them securely.

### Goals
- Expose existing REST APIs as tools for agents via the Model Context Protocol (MCP).
- Understand how to package a tool for deployment on AgentCore Runtime.
- Configure authentication for tools (OAuth2 credential providers and/or IAM-based access).
- Deploy custom MCP tools using the `agentcore_starter_toolkit` and test them with existing agents.
- Learn how to store secrets and client credentials securely and reuse them.



### Key tools & libraries covered
- [AgentCore Gateway](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html) to securely expose tools to agents
- [AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) to host and run tools
- [AgentCore Identity](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/identity.html) to securely manage authentication and authorization
- [Cognito OAuth2 provider](https://docs.aws.amazon.com/cognito/latest/developerguide/what-is-amazon-cognito.html) for OAuth2-based authentication
- [bedrock_agentcore_starter_toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit) to simplify building, packaging, and deploying tools
- [AgentCore Control Plane API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control.html) used to deploy AgentCore resources such as agents and tools programmatically
- [AgentCore DataPlane API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore.html) used to invoke AgentCore resources programmatically
- [Model Context Protocol (MCP)](https://modelcontextprotocol.io/docs/getting-started/intro) a protocol used to standardize tool invocation and responses
- [MCP Python SDK](https://github.com/modelcontextprotocol/python-sdk) Official Python MCP SDK to simplify building MCP-compliant servers and clients


### Expose existing APIs as MCP Tools using AgentCore Gateway

We saw in the prior notebook how to create custom tools for our Strands Agent by decorating a Python function with the `@tool` decorator. While this is a great way to quickly prototype tools in a notebook environment, it comes with several limitations:
- The tool is only available within the context of that specific notebook session
- The tool cannot be easily shared or reused across multiple agents or users
- The tool only works with the Strands Agents framework

We can address these limitations by deploying our custom tools as MCP Tools using AgentCore Gateway. Normally we would use an [MCP SDK](https://github.com/modelcontextprotocol/python-sdk) to wrap our existing APIs as MCP Tools, but AgentCore Gateway provides a simplified way to do this by directly integrating with existing REST HTTP APIs.

We'll use the [agentcore_starter_toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit) which provides a set of pre-built tools to simplify the deployment process

In [ ]:
%pip install -Uq -r requirements.txt 2> /dev/null && echo "All required packages are installed."

In [ ]:
import boto3
from pathlib import Path
import json
from bedrock_agentcore_starter_toolkit.operations.gateway import GatewayClient

In [3]:
REGION = boto3.Session().region_name

# Used to retrieve client secret for Cognito OAuth flow
SECRETS_MANAGER_CLIENT = boto3.client("secretsmanager")

# this json contains various workshop resource ARNs and endpoints that we will need
# these have been pre-populated for you as part of workshop setup
WORKSHOP_RESOURCES = json.loads(Path("workshop_resources.json").read_text())

### AgentCore Gateway Key Concepts
There are a number of key concepts to understand when working with AgentCore Gateway:
- **Gateway**: The main resource that will expose an MCP endpoint that will be used to discover and invoke tools
- **Target**: A backend service that implements the actual tool logic. This can be an existing REST API, a Lambda function, or another MCP endpoint
- **Tool**: A specific function or capability that is exposed via the Gateway. 
- **Inbound Authentication**: The method used to authenticate requests coming into the Gateway from agents. Note that the MCP spec calls for OAuth2 based authentication.
- **Outbound Authentication**: The method used to authenticate requests going out from the Gateway to the Target service. This can include API keys, IAM roles, etc.

In [4]:
# The GatewayClient provides abstraction over the low-level API calls
gateway_client = GatewayClient(region_name=REGION) 

For the initial example, we will convert the geocoding tool we built in the prior notebook into an MCP Tool deployed via AgentCore Gateway. 

The first step is to create the Gateway which will call `AddressSearchTools` and configure it to use OAuth2 based inbound authentication using Amazon Cognito.
The Cognito User Pool and App Client have already been created for us as part of the workshop setup. We will use an [M2 (machine-to-machine)](https://docs.aws.amazon.com/cognito/latest/developerguide/cognito-user-pools-define-resource-servers.html#cognito-user-pools-define-resource-servers-about-m2m) flow to authenticate the agent when it calls into the Gateway.
This will allow for the Agent to obtain credentials directly, we can also configure a user-based flow if desired that will prompt for user login.

In [5]:
address_search_gateway_name = "AddressSearchTools"

# Check if the gateway already exists
existing_gateways = [
    g
    for g in gateway_client.client.list_gateways()["items"]
    if g["name"] == address_search_gateway_name
]

if existing_gateways:
    print(f"Gateway {address_search_gateway_name} already exists.")
    address_search_gateway = existing_gateways[0]
    
# Create the Gateway if it does not exist    
else:
    address_search_gateway = gateway_client.create_mcp_gateway(
        name=address_search_gateway_name,
        role_arn=WORKSHOP_RESOURCES["AgentCoreExecutionRole"],

        # use the Cognito User Pool for OAuth2 based authentication
        authorizer_config={
            "customJWTAuthorizer": {
                "discoveryUrl": WORKSHOP_RESOURCES["CognitoDiscoveryUrl"],
                "allowedClients": [
                    WORKSHOP_RESOURCES["CognitoClientId"],
                ],
            }
        },
    )

2026-02-19 21:09:31,793 - bedrock_agentcore.gateway - INFO - Creating Gateway
2026-02-19 21:09:31,933 - bedrock_agentcore.gateway - INFO - ✓ Created Gateway: arn:aws:bedrock-agentcore:us-west-2:780660052610:gateway/addresssearchtools-d0gsfabzag
2026-02-19 21:09:31,934 - bedrock_agentcore.gateway - INFO -   Gateway URL: https://addresssearchtools-d0gsfabzag.gateway.bedrock-agentcore.us-west-2.amazonaws.com/mcp
2026-02-19 21:09:31,935 - bedrock_agentcore.gateway - INFO -   Waiting for Gateway to be ready...
2026-02-19 21:09:34,128 - bedrock_agentcore.gateway - INFO - 
✅Gateway is ready


The next step is to create the Target. There are numerous supported Targets including REST APIs, Lambda functions, and MCP endpoints. In our case we will create a REST API Target that points to the existing OpenStreet Map Nominatim API we used in the prior notebook. To register an HTTP Target we need to:
- Provide the OpenAPI spec for the API (These are usually provided by the [API owner](https://nominatim.org/release-docs/latest/api/Search/), but can also be created manually)
- Provide the credentials needed to access the API (in this case no authentication is required)

On the second point, the OpenStreet Map Nominatim API is a public API that does not require authentication. However, it does require that clients provide a valid `User-Agent` header with each request. We can use this as a proxy for authentication by configuring the Gateway to inject this header into each outbound request. This would work similar for a Bearer token or API key if the target API required one.


In [6]:
open_street_map_target_name = "OpenStreetMap"

# Schema for the api
# Each path will be exposed as a separate tool in the Gateway
open_street_map_api_schema = {
    "openapi": "3.0.0",
    "info": {
        "title": "Nominatim Address Search API",
        "description": "OpenStreetMap Nominatim API for address geocoding and search",
        "version": "1.0.0",
    },
    "servers": [
        {
            "url": "https://nominatim.openstreetmap.org",
            "description": "OpenStreetMap Nominatim API",
        }
    ],
    "paths": {
        "/search": {
            "get": {
                "operationId": "searchAddress",
                "summary": "Search for an address",
                "description": "Retrieves address information and geocoding data from OpenStreetMap",
                "parameters": [
                    {
                        "name": "q",
                        "in": "query",
                        "description": "The address to search for",
                        "required": True,
                        "schema": {"type": "string"},
                    },
                    {
                        "name": "format",
                        "in": "query",
                        "required": True,
                        "schema": {
                            "type": "string",
                            "enum": ["json"],
                            "default": "json",
                        },
                    },
                    {
                        "name": "addressdetails",
                        "in": "query",
                        "schema": {"type": "integer", "enum": [0, 1], "default": 1},
                    },
                    {
                        "name": "limit",
                        "in": "query",
                        "schema": {"type": "integer", "default": 5},
                    },
                    {
                        "name": "countrycodes",
                        "in": "query",
                        "schema": {"type": "string"},
                    },
                ],
                "responses": {"200": {"description": "Successful response"}},
            }
        }
    },
}


# check if the target already exists
gateway_targets = gateway_client.client.list_gateway_targets(
        gatewayIdentifier=address_search_gateway["gatewayId"]
    )["items"]


if gateway_targets:
    print(f"Gateway target {open_street_map_target_name} already exists.")
    open_street_map_target = [
        t for t in gateway_targets if t["name"] == open_street_map_target_name
    ][0]

else:
    open_street_map_target = gateway_client.create_mcp_gateway_target(
        gateway=address_search_gateway,
        name=open_street_map_target_name,
        target_type="openApiSchema",
        target_payload={"inlinePayload": json.dumps(open_street_map_api_schema)},

        # credentials will be securely stored in AgentCore Credential Vault
        # you can configure to inject credentials into headers, query parameters, or body depending on the target API requirements
        credentials={
            "api_key": "PropertyResearchBot/1.0 (contact@agentic.com)",
            "credential_location": "HEADER",
            "credential_parameter_name": "USER-AGENT",
        },
    )

2026-02-19 21:10:39,572 - bedrock_agentcore.gateway - INFO - Creating credential provider
2026-02-19 21:10:40,001 - bedrock_agentcore.gateway - INFO - ✓ Added credential provider successfully (ARN: arn:aws:bedrock-agentcore:us-west-2:780660052610:token-vault/default/apikeycredentialprovider/OpenStreetMap-ApiKey-3d4a6eb6)
2026-02-19 21:10:40,004 - bedrock_agentcore.gateway - INFO - Creating Target
2026-02-19 21:10:40,006 - bedrock_agentcore.gateway - INFO - {'gatewayIdentifier': 'addresssearchtools-d0gsfabzag', 'name': 'OpenStreetMap', 'targetConfiguration': {'mcp': {'openApiSchema': {'inlinePayload': '{"openapi": "3.0.0", "info": {"title": "Nominatim Address Search API", "description": "OpenStreetMap Nominatim API for address geocoding and search", "version": "1.0.0"}, "servers": [{"url": "https://nominatim.openstreetmap.org", "description": "OpenStreetMap Nominatim API"}], "paths": {"/search": {"get": {"operationId": "searchAddress", "summary": "Search for an address", "description": 

Now we can invoke the tool via the Gateway's MCP endpoint! But first we need to obtain an access token from Cognito to authenticate our request.

In [7]:
# gateway url will always be in the format below
gateway_url = f"https://{address_search_gateway['gatewayId']}.gateway.bedrock-agentcore.{REGION}.amazonaws.com/mcp"

# client secret for the Cognito App Client used for M2M authentication
client_secret = SECRETS_MANAGER_CLIENT.get_secret_value(
    SecretId=WORKSHOP_RESOURCES["CognitoClientSecretArn"]
)["SecretString"]

# obtain OAuth2 token from Cognito
oauth2_token = gateway_client.get_access_token_for_cognito(
    client_info={
        "client_id": WORKSHOP_RESOURCES["CognitoClientId"],
        "client_secret": client_secret,
        "scope": "gateway/invoke",
        "token_endpoint": WORKSHOP_RESOURCES["CognitoTokenEndpoint"],
    }
)

2026-02-19 21:11:13,338 - bedrock_agentcore.gateway - INFO - Fetching test token from Cognito...
2026-02-19 21:11:13,339 - bedrock_agentcore.gateway - INFO -   Attempting to connect to token endpoint: https://agentcore-m2m-domain-780660052610-us-west-2.auth.us-west-2.amazoncognito.com/oauth2/token
2026-02-19 21:11:13,895 - bedrock_agentcore.gateway - INFO - ✓ Got test token successfully


Let's recreate the primary residence mortgage verification agent from the prior notebook, but this time using the OpenStreet Map tool deployed via AgentCore Gateway.

In [8]:
from strands import Agent

from strands.models import BedrockModel
from strands_tools.calculator import calculator

from strands.tools.mcp import MCPClient #<- New import MCP Client to connect to remote hosted tools
from mcp.client.streamable_http import streamablehttp_client

MODEL =  BedrockModel(
    model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    temperature=0.0,
)

Tools that are exposed via remote MCP endpoints are configured differently than local tools. Instead of using the `@tool` decorator, we create an `MCPClient` that establishes a connection to the remote MCP endpoint and can be used to discover and invoke tools. For more details, refer to the [Strands MCP Documentation](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/).

In [9]:

# create MCP client to connect to the Gateway
address_search_mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway_url, headers={"Authorization": f"Bearer {oauth2_token}"}
    )
)

# Use the MCP client to list and call remote tools
# The `with` context block will handle opening and closing the connection
with address_search_mcp_client:

    remote_mcp_tools = address_search_mcp_client.list_tools_sync()

    PRIMARY_RESIDENCE_MORTGAGE_VERIFICATION_PROMPT = """
    You are a mortgage verification agent whose job is to verify if a mortgage applicant is eligible for a primary residence mortgage.
    In order to qualify for a primary residence mortgage, the applicant must meet the following criteria:
    1. The property must be in the same country as the applicant's job location.
    2. The property must be within 30 miles of the applicant's job location. Euclidean distance is acceptable for this calculation.
    3. The applicant must not own any other properties.

    Use the tools at your disposal to gather the necessary information to make a decision.
    """

    primary_residence_verification_agent = Agent(
        model=MODEL,
        tools=remote_mcp_tools + [calculator],
        system_prompt=PRIMARY_RESIDENCE_MORTGAGE_VERIFICATION_PROMPT,
    )

    response = primary_residence_verification_agent(
        "The applicant's work location is Venetian, Las Vegas and they are applying for a mortgage at 2928 Coast Line Ct, Las Vegas, NV 89117."
    )

I'll help you verify the mortgage applicant's eligibility. Let me gather the necessary information by looking up both the work location and the property location.
Tool #1: OpenStreetMap___searchAddress

Tool #2: OpenStreetMap___searchAddress
Great! I found both locations. Now let me calculate the distance between them:

**Work Location (The Venetian Las Vegas):**
- Latitude: 36.1217073
- Longitude: -115.1693352

**Property Location (2928 Coast Line Ct):**
- Latitude: 36.1373712
- Longitude: -115.2879006

Let me calculate the Euclidean distance:
Tool #3: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬──────────────────────────────────────────────────────────────────────────────╮                   │
│  │ Operation │ Evaluate Expression                                                          │                   │
│  │ Input     │ sqrt((36.1373712 - 36.1217073)**2 + (-115.2879006 - (-115.1693352))**2) * 69 │                   │
│  │ Result    │ 8.25                                                                         │                   │
│  ╰───────────┴──────────────────────────────────────────────────────────────────────────────╯                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Perfect! Now let me verify the eligibility criteria:

## Mortgage Eligibility Verification

**Applicant Information:**
- Work Location: The Venetian Las Vegas, Nevada
- Property Address: 2928 Coast Line Ct, Las Vegas, NV 89117

**Eligibility Criteria Assessment:**

✅ **Criterion 1: Same Country**
- Work location country: United States
- Property location country: United States
- **Status: PASS** - Both are in the same country

✅ **Criterion 2: Within 30 Miles of Work Location**
- Distance calculated: **8.25 miles**
- Maximum allowed: 30 miles
- **Status: PASS** - Property is well within the 30-mile radius

⚠️ **Criterion 3: No Other Properties Owned**
- **Status: UNABLE TO VERIFY** - I do not have access to property ownership records for this applicant. You will need to verify this through:
  - Credit report review
  - Property ownership database search
  - Applicant's financial disclosure forms
  - County assessor records

## Conclusion

The applicant **meets 2 out of 3 criteria** tha

### Deploy Custom Tools via AgentCore Runtime
With the AgentCore Runtime we can deploy agents or custom tools in a managed environment that can scale to support multiple users and agents. 

Recall that for the KYC Agent in the prior notebook, we built a number of custom tools to lookup customer information across multiple data sources. Unlike the address search tool, these tools do not have existing REST APIs so we need a different deployment approach to expose them as MCP Tools. This is where the AgentCore Runtime comes in, allowing us to package these tools as an MCP microservice that can be easily invoked by the agent.

Using the official [MCP SDK](https://modelcontextprotocol.io/docs/getting-started/intro), we can wrap the existing python functions as MCP Tools and deploy them to the AgentCore Runtime. You can view the source code for the custom tools [here](workspace/kyc_tools/mcp_server.py)

To deploy tools or Agents to the AgentCore Runtime, we need to follow these steps:
1. Package the tool or agent code to an ARM64 compatible Docker container
2. Upload the container to Amazon ECR
3. Deploy the container to AgentCore Runtime using [AgentCore Control Plane APIs](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agentcore-control.html)

Fortunately, the `agentcore_starter_toolkit` provides helper functions to simplify this process by wrapping all of the required deployment steps into a single function call.

In [10]:
from bedrock_agentcore_starter_toolkit.operations.runtime import (
    configure_bedrock_agentcore,
    launch_bedrock_agentcore,
)

For simplicity, we will copy the synthetic data into the container image itself. In a production environment, you would typically connect to external data sources such as databases or APIs.

In [11]:
!cp -r synthetic_data/ workspace/kyc_tools/

The `configure_bedrock_agentcore` function allows us to customize the deployment. It will generate a yaml based configuration file that can then be used to deploy the agent.

In [15]:
mcp_tool_configuration = configure_bedrock_agentcore(
    agent_name="kyc_mcp_tooling",
    protocol="MCP",
    execution_role=WORKSHOP_RESOURCES["AgentCoreExecutionRole"],    # this role will be assumed by the runtime instance
    source_path="workspace/kyc_tools",                              # files in this directory will be packaged into a Docker image
    entrypoint_path=Path("workspace/kyc_tools/mcp_server.py"),      # this script will be executed when the container starts
    non_interactive=True,
    
    # we'll use the same Cognito based authorizer as the Gateway
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [WORKSHOP_RESOURCES["CognitoClientId"]],
            "discoveryUrl": WORKSHOP_RESOURCES["CognitoDiscoveryUrl"],
        }
    },
    code_build_execution_role=WORKSHOP_RESOURCES["CodeBuildRole"],   # role for building the Docker image
    memory_mode="NO_MEMORY",                                         # we are not saving any state so memory is not needed
    verbose=True,
)

Verbose mode enabled
Configuring BedrockAgentCore agent: kyc_mcp_tooling
Build directory: /home/sagemaker-user/strands-agentcore-workshop
Source path: workspace/kyc_tools
Bedrock AgentCore name: kyc_mcp_tooling
Entrypoint path: workspace/kyc_tools/mcp_server.py
Retrieving AWS account information...
AWS account ID: 780660052610
AWS region: us-west-2
Initializing container runtime with: default
Using execution role: arn:aws:iam::780660052610:role/BedrockAgentCoreRole
Memory disabled
Using separate CodeBuild execution role: arn:aws:iam::780660052610:role/CodeBuildExecutionRole
Attempting to find Bedrock AgentCore instance name in workspace/kyc_tools/mcp_server.py
Generating Dockerfile with parameters:
  Entrypoint: workspace/kyc_tools/mcp_server.py
  Build directory: /home/sagemaker-user/strands-agentcore-workshop
  Bedrock AgentCore name: bedrock_agentcore
  Region: us-west-2
  Enable observability: True
  Requirements file: None


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: .bedrock_agentcore/kyc_mcp_tooling/Dockerfile
Agent name from BedrockAgentCoreApp: kyc_mcp_tooling
Config path: /home/sagemaker-user/strands-agentcore-workshop/.bedrock_agentcore.yaml
Using entrypoint format: workspace/kyc_tools/mcp_server.py
ECR auto-create: True
Creating BedrockAgentCoreConfigSchema with following parameters:
  Name: kyc_mcp_tooling
  Entrypoint: workspace/kyc_tools/mcp_server.py
  Platform: linux/arm64
  Container runtime: docker
  Execution role: arn:aws:iam::780660052610:role/BedrockAgentCoreRole
  ECR repository: Auto-create
  Enable observability: True
  Request header configuration: None
Keeping 'kyc_mcp_tooling' as default agent
Configuration saved with agent: kyc_mcp_tooling


In [13]:
# this function will build the Docker image, push it to ECR, and launch the AgentCore runtime instance
launch_mcp_server = launch_bedrock_agentcore(config_path=mcp_tool_configuration.config_path, auto_update_on_conflict=True)

Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'kyc_mcp_tooling' to account 780660052610 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: kyc_mcp_tooling
✅ ECR repository available: 780660052610.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-kyc_mcp_tooling
Using execution role from config: arn:aws:iam::780660052610:role/BedrockAgentCoreRole
Preparing CodeBuild project and uploading source...


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-kyc_mcp_tooling


Using CodeBuild role from config: arn:aws:iam::780660052610:role/CodeBuildExecutionRole
Created S3 bucket: bedrock-agentcore-codebuild-sources-780660052610-us-west-2
Using dockerignore.template with 45 patterns for zip filtering
Including Dockerfile from /home/sagemaker-user/strands-agentcore-workshop/.bedrock_agentcore/kyc_mcp_tooling in source.zip
Uploaded source to S3: kyc_mcp_tooling/source.zip
Created CodeBuild project: bedrock-agentcore-kyc_mcp_tooling-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.3s
🔄 DOWNLOAD_SOURCE started (total: 10s)
✅ DOWNLOAD_SOURCE completed in 1.0s
🔄 BUILD started (total: 11s)
✅ BUILD completed in 23.8s
🔄 POST_BUILD started (total: 35s)
✅ POST_BUILD completed in 20.7s
🔄 COMPLETED started (total: 56s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 56s
CodeBui

In [16]:
# MCP servers deployed to the runtime will always have URLs in the format below
# Characters in the ARN must be URL encoded
encoded_arn = launch_mcp_server.agent_arn.replace(':', '%3A').replace('/', '%2F')
kyc_mcp_url = f"https://bedrock-agentcore.{REGION}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"

The KYC Agent from lab 1 can now be updated to use the deployed tools instead of the local implementations.

In [17]:
KYC_RESEARCH_AGENT_PROMPT = """
You are a Know Your Customer research agent whose job is to gather information about mortgage applicants to help determine their mortgage eligibility.
You have access to the following tools:
1. Credit Report Search: Use this tool to search credit report data by fields 'full_legal_name' or 'primary_address'.
2. Income Verification Search: Use this tool to search income verification data by fields 'employee_name'.
3. Property Records Search: Use this tool to search property records data by fields 'owner_name_on_deed' or 'property_address'.
4. Lien Records Search: Use this tool to search lien records data by fields 'debtor_name' or 'debtor_address'.
5. Calculator: Use this tool to perform mathematical calculations, including distance calculations.

Gather the following information about the applicant:
1. Full legal name
2. Current address
3. Employment status and income
4. Credit score
5. Debt to income ratio
6. Any liens or judgments against them

Note that the applicant's name and address may be represented in different ways in different datasets. 
Use fuzzy matching and partial matches across datasets to ensure you find all relevant information.

Use the tools at your disposal to gather the necessary information to help support the mortgage verification decision.
"""

# re-obtain OAuth2 token from Cognito
oauth2_token = gateway_client.get_access_token_for_cognito(
    client_info={
        "client_id": WORKSHOP_RESOURCES["CognitoClientId"],
        "client_secret": client_secret,
        "scope": "gateway/invoke",
        "token_endpoint": WORKSHOP_RESOURCES["CognitoTokenEndpoint"],
    }
)

kyc_mcp_client = MCPClient(lambda : streamablehttp_client(kyc_mcp_url, headers={"Authorization": f"Bearer {oauth2_token}"}))

with kyc_mcp_client:
    
    kyc_tools = kyc_mcp_client.list_tools_sync()
    
    kyc_research_agent = Agent(
        model=MODEL,
        tools=[
            *kyc_tools,
            calculator,
        ],
        system_prompt=KYC_RESEARCH_AGENT_PROMPT,
    )

    kyc_response = kyc_research_agent("Gather information about applicant William Mcgee")
    
    # if you see a session termination failed: 404 error, its a known issue with the current version of the MCP setup and can be ignored for now.


2026-02-19 21:18:20,163 - bedrock_agentcore.gateway - INFO - Fetching test token from Cognito...
2026-02-19 21:18:20,165 - bedrock_agentcore.gateway - INFO -   Attempting to connect to token endpoint: https://agentcore-m2m-domain-780660052610-us-west-2.auth.us-west-2.amazoncognito.com/oauth2/token
2026-02-19 21:18:20,645 - bedrock_agentcore.gateway - INFO - ✓ Got test token successfully


I'll gather information about William Mcgee by searching across multiple databases. Let me start with credit reports, income verification, property records, and lien records.
Tool #1: credit_report_search

Tool #2: income_verification_search

Tool #3: property_records_search

Tool #4: lien_records_search
Great! I found a match for William Mcgee. Let me calculate the debt-to-income ratio and compile all the information.
Tool #5: calculator

Tool #6: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬───────────────────────────────────────────────────────────────╮                                  │
│  │ Operation │ Evaluate Expression                                           │                                  │
│  │ Input     │ 3980.41 + 739.41 + 275.82 + 530.82 + 493.98 + 113.23 + 803.02 │                                  │
│  │ Result    │ 6936.6899999976                                               │                                  │
│  ╰───────────┴───────────────────────────────────────────────────────────────╯                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬──────────────────────────────────────────────────────────────────────────────────────────╮       │
│  │ Operation │ Evaluate Expression                                                                      │       │
│  │ Input     │ (3980.41 + 739.41 + 275.82 + 530.82 + 493.98 + 113.23 + 803.02) / (171461.41 / 12) * 100 │       │
│  │ Result    │ 48.56                                                                                    │       │
│  ╰───────────┴──────────────────────────────────────────────────────────────────────────────────────────╯       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Summary of Information for William Mcgee

### Personal Information
- **Full Legal Name:** William Mcgee
- **Government ID:** 878-64-7325
- **Date of Birth:** January 1, 1956
- **Current Address:** 801 Wu Points, Lake Emily, MA 48125

### Employment Status & Income
- **Employer:** Velasquez Group
- **Verified Annual Salary:** $171,461.41
- **Monthly Income:** $14,288.45

### Credit Information
- **Credit Score:** 712 (Fair/Good)
- **Account Tradelines:**
  - Mortgage: $724,434.99 balance, $3,980.41/month
  - HELOC: $88,729.02 balance, $739.41/month
  - Auto Loan: $9,929.41 balance, $275.82/month
  - HELOC: $63,698.94 balance, $530.82/month
  - HELOC: $59,277.29 balance, $493.98/month
  - Credit Card: $2,680.42 balance (27% utilization), $113.23/month
  - Mortgage: $277,042.17 balance, $803.02/month

### Debt Analysis
- **Total Monthly Debt Payments:** $6,936.69
- **Debt-to-Income Ratio:** 48.56%

### Property Information
- **Property Address:** 801 Wu Points, Lake Emily, MA 48125
- *

Session termination failed: 404


We'll save the information associated with the deployed tools to a `lab_resources.json` file so we can use them in the next lab.

In [19]:
try:
    with open("lab_resources.json", "r") as f:
        data = json.load(f)
except FileNotFoundError:
    data = {}

# Add or update the keys
data.update({
    "GatewayUrl": gateway_url,
    "GatewayName": address_search_gateway_name,
    "GatewayId": address_search_gateway["gatewayId"],
    "OpenStreetMapTargetId": open_street_map_target["targetId"],
    "KYC_MCP_Runtime_Url": kyc_mcp_url,
    "KYC_MCP_Runtime_Id": launch_mcp_server.agent_id,
})

# Write the updated data back to the file
with open("lab_resources.json", "w") as f:
    json.dump(data, f, indent=4)

### Conclusion
In this lab we learned how to deploy existing REST APIs as MCP Tools using AgentCore Gateway, and how to package and deploy custom tools using AgentCore Runtime. We also saw how to configure authentication for tools using OAuth2 and Cognito. In the next lab, we will explore how to deploy and monitor our agent with AgentCore Runtime and AgentCore Observability.